# Instalações das bibliotecas

In [ ]:
!pip install python-telegram-bot==13.7
!pip install llama_cpp_python
!pip install pytelegrambotapi

# LLama (Inteligência Artificial)

Configuração da IA

In [ ]:
from llama_cpp import Llama
from llama_cpp.llama_tokenizer import LlamaHFTokenizer

llm = Llama.from_pretrained(
    repo_id="meetkai/functionary-small-v2.4-GGUF",
    filename="functionary-small-v2.4.Q4_0.gguf",
    chat_format="functionary-v2",
    tokenizer=LlamaHFTokenizer.from_pretrained("meetkai/functionary-small-v2.4-GGUF"),
    n_gpu_layers=-1,
)

# Funções do Chatbot

Definindo as funções

In [ ]:
import sqlite3

# Funções
def greet_user():
    return "Olá! Como posso ajudar você hoje?"

def search_books(name):
    conn = sqlite3.connect('bookstore.db', timeout=10)
    cursor = conn.cursor()

    query = "SELECT title FROM books WHERE title LIKE ?"
    cursor.execute(query, ('%' + name + '%',))
    books = cursor.fetchall()

    conn.close()

    if books:
        return "Livros encontrados:\n" + "\n".join(f"* {book[0]}" for book in books)
    else:
        return "Nenhum livro encontrado com esse nome."

# Estrutura para armazenar o carrinho
cart = []

def add_to_cart(book_title):
    conn = sqlite3.connect('bookstore.db', timeout=10)
    cursor = conn.cursor()

    query = "SELECT title, price FROM books WHERE title = ?"
    cursor.execute(query, (book_title,))
    book = cursor.fetchone()

    conn.close()

    if book:
        cart.append(book)
        return f"{book_title} foi adicionado ao seu carrinho."
    else:
        return "Livro não encontrado."

def remove_from_cart(book_title):
    for book in cart:
        if book[0] == book_title:
            cart.remove(book)
            return f"{book_title} foi removido do seu carrinho."
    return "Livro não encontrado no seu carrinho."

def view_cart():
    if cart:
        total = sum(book[1] for book in cart)
        books_in_cart = "\n".join(f"* {book[0]} - R${book[1]:.2f}" for book in cart)
        return f"Livros no seu carrinho:\n{books_in_cart}\n\nTotal: R${total:.2f}"
    else:
        return "Seu carrinho está vazio."

def checkout():
    if cart:
        total = sum(book[1] for book in cart)
        cart.clear()  # Esvaziar o carrinho após a compra
        return f"Sua compra foi finalizada com sucesso. Total: R${total:.2f}"
    else:
        return "Seu carrinho está vazio."

def get_author_by_book_title(book_title):
    conn = sqlite3.connect('bookstore.db', timeout=10)
    cursor = conn.cursor()

    query = '''
        SELECT authors.name
        FROM books
        JOIN authors ON books.author_id = authors.id
        WHERE books.title = ?
    '''
    cursor.execute(query, (book_title,))
    author = cursor.fetchone()

    conn.close()

    if author:
        return f"O autor do livro '{book_title}' é {author[0]}."
    else:
        return "Autor não encontrado para esse livro."

def list_all_books():
    conn = sqlite3.connect('bookstore.db', timeout=10)
    cursor = conn.cursor()

    query = "SELECT title FROM books"
    cursor.execute(query)
    books = cursor.fetchall()

    conn.close()

    if books:
        return "Lista de todos os livros:\n" + "\n".join(f"* {book[0]}" for book in books)
    else:
        return "Nenhum livro encontrado no banco de dados."


# Mapeamento das funções
functions_map = {
    "greet_user": greet_user,
    "search_books": search_books,
    "add_to_cart": add_to_cart,
    "remove_from_cart": remove_from_cart,
    "view_cart": view_cart,
    "checkout": checkout,
    "get_author_by_book_title": get_author_by_book_title,
    "list_all_books":list_all_books,
}

# Ferramentas do chatbot
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_books",
            "description": "Search for a specific book",
            "parameters": {
                "type": "object",
                "properties": {"name": {"type": "string", "description": "The name of the book"}},
                "required": ["name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "greet_user",
            "description": "Greet the user and ask how to help",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "add_to_cart",
            "description": "Add a book to the cart",
            "parameters": {
                "type": "object",
                "properties": {
                    "book_title": {"type": "string", "description": "The title of the book to add"},
                },
                "required": ["book_title"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "remove_from_cart",
            "description": "Remove a book from the cart",
            "parameters": {
                "type": "object",
                "properties": {
                    "book_title": {"type": "string", "description": "The title of the book to remove"},
                },
                "required": ["book_title"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "view_cart",
            "description": "View the prices of books in the cart and the total amount",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "checkout",
            "description": "Checkout and finalize the purchase",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_author_by_book_title",
            "description": "Get the author of a specific book",
            "parameters": {
                "type": "object",
                "properties": {
                    "book_title": {"type": "string", "description": "The title of the book"},
                },
                "required": ["book_title"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "list_all_books",
            "description": "List all books in the database",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    }
]


# Banco SQLite

Configuração do Banco SQLite

In [ ]:
import sqlite3

conn = sqlite3.connect('bookstore.db', timeout=10)
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS books')
cursor.execute('DROP TABLE IF EXISTS authors')

# Criando a tabela authors
cursor.execute('''
CREATE TABLE IF NOT EXISTS authors (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
)
''')

# Criando a tabela books
cursor.execute('''
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author_id INTEGER,
    price REAL,
    FOREIGN KEY (author_id) REFERENCES authors (id)
)
''')

# Inserindo autores
authors = [
    ('J.K. Rowling',),
    ('J.R.R. Tolkien',),
    ('George Orwell',),
    ('Stephen King',),
    ('Agatha Christie',),
    ('Jules Verne',),
    ('Leo Tolstoy',),
    ('Jane Austen',),
    ('Ernest Hemingway',),
    ('F. Scott Fitzgerald',),
    ('Arthur Conan Doyle',),
    ('Emily Bronte',),
    ('Harper Lee',),
    ('Monteiro Lobato',),
    ('Machado de Assis',),
    ('Carlos Drummond de Andrade',),
    ('Clarice Lispector',),
    ('Graciliano Ramos',),
    ('Érico Veríssimo',),
]
cursor.executemany('INSERT INTO authors (name) VALUES (?)', authors)

# Inserindo livros
books = [
    ('Harry Potter e a Pedra Filosofal', 1, 19.99),
    ('Harry Potter e a Câmara Secreta', 1, 24.99),
    ('O Hobbit', 2, 15.99),
    ('1984', 3, 12.99),
    ('O Iluminado', 4, 20.50),
    ('Assassinato no Expresso do Oriente', 5, 18.75),
    ('Viagem ao Centro da Terra', 6, 14.99),
    ('Guerra e Paz', 7, 22.99),
    ('Orgulho e Preconceito', 8, 16.50),
    ('O Velho e o Mar', 9, 17.25),
    ('O Grande Gatsby', 10, 21.00),
    ('As Aventuras de Huckleberry Finn', 11, 19.25),
    ('O Morro dos Ventos Uivantes', 12, 15.99),
    ('A Guerra dos Tronos', 13, 23.75),
    ('Para Matar um Mockingbird', 14, 18.50),
    ('Dom Casmurro', 15, 17.99),
    ('Memórias Póstumas de Brás Cubas', 16, 22.50),
    ('A Rosa do Povo', 17, 26.00),
    ('A Hora da Estrela', 18, 17.99),
    ('Vidas Secas', 19, 19.75),
]
cursor.executemany('INSERT INTO books (title, author_id, price) VALUES (?, ?, ?)', books)

# Salvando (commit) as alterações e fechando a conexão
conn.commit()
conn.close()


# Chatbot

Configuração do Chatbot

In [ ]:
import telebot
import json

# Altere para o token do seu bot
API_TOKEN = '7256091217:AAEtoFh0pWVfj6V2x-dUZu9M4JSK-aGIaBI'

bot = telebot.TeleBot(API_TOKEN)

def get_function_details(llm, message, tools):
    result = llm.create_chat_completion(
        messages=[{"role": "user", "content": message}],
        tools=tools,
        tool_choice="auto",
    )
    function_call = result["choices"][0]["message"]["tool_calls"][0]["function"]
    function_name = function_call["name"]
    arguments = json.loads(function_call["arguments"])
    return function_name, arguments

def execute_function(function_name, arguments, functions_map):
    function = functions_map.get(function_name)
    if function:
        return function(**arguments)
    return f"Deculpe, eu não entendi o que você quis dizer."

@bot.message_handler(func=lambda message: True)
def reply_to_message(message):
    function_name, arguments = get_function_details(llm, message.text, tools)
    response = execute_function(function_name, arguments, functions_map)
    bot.reply_to(message, response)

Ativação do Chatbot

In [6]:
bot.polling()